In [1]:
!pip install tensorflow
!pip install matplotlib
import tensorflow as tf
from tensorflow.keras import models,layers
import matplotlib.pyplot as plt
from tensorflow import keras
import keras.layers
from PIL import Image
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score,recall_score,precision_score
import numpy as np

In [2]:
import zipfile
import os
zip_file_path = './mrismall800.zip'
extract_dir = './mrismall800'
if not os.path.exists(extract_dir):
    os.makedirs(extract_dir)
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)
print(os.listdir(extract_dir))

['mri']


In [3]:
IMAGE_SIZE=650
BATCH_SIZE=32

In [4]:
import os
print(os.listdir('./mrismall800/mri/dataset'))

['Normal', 'Stroke']


In [5]:
df=tf.keras.preprocessing.image_dataset_from_directory(
    "mrismall800/mri/dataset",
    shuffle=True,
    image_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

Found 800 files belonging to 2 classes.


In [6]:
class_names=df.class_names
class_names

['Normal', 'Stroke']

In [7]:
image_count=0
for images, labels in df:
    image_count += images.shape[0]

In [8]:
image_count

800

In [9]:
def  total_images(dataset):
    count = 0
    for _ in dataset:
        count += 1
    return count

In [10]:
def split(df,train_split=0.75,test_split=0.25,shuffle=True,shuffle_size=5000):
  df_shape= total_images(df.unbatch())
  df = df.unbatch().batch(1)
  if shuffle:
    df=df.shuffle(shuffle_size,seed=50)
  train_size=int(train_split*df_shape)
  test_size=int(test_split*df_shape)
  train_df=df.take(train_size)
  test_df=df.skip(train_size).take(test_size)
  return train_df,test_df


In [11]:
train_imgs,test_imgs=split(df)

In [12]:
resize_rescale=tf.keras.Sequential([
    layers.Resizing(650,650),
    layers.Rescaling(1.0/255)
])
data_enhance = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.3),
    layers.RandomZoom(0.3)
])

In [16]:
def eleven_layer_cnn_softmax_kernel5():
  model = keras.models.Sequential([
      resize_rescale,
    data_enhance,
    keras.layers.Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=(650,650,3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(filters=64, kernel_size=(5,5), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(filters=128, kernel_size=(5,5), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(2, activation='softmax')
      ])
  model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model


In [17]:
train_labels = np.concatenate([y.numpy() for _, y in train_imgs], axis=0)
test_labels = np.concatenate([y.numpy() for _, y in test_imgs], axis=0)

In [ ]:
model1=eleven_layer_cnn_softmax_kernel5()
model1.fit(train_imgs,epochs=10)
y_pred_probs = model1.predict(test_imgs.map(lambda x, y: x))
y_pred = np.argmax(y_pred_probs, axis=1)
accuracy = accuracy_score(test_labels, y_pred)
f1 = f1_score(test_labels, y_pred)
precision = precision_score(test_labels, y_pred)
recall = recall_score(test_labels, y_pred)
print("Model 1")
print(accuracy)
print(f1)
print(precision)
print(recall)

Epoch 1/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 745s 1s/step - accuracy: 0.7007 - loss: 1.3980
Epoch 2/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 665s 1s/step - accuracy: 0.7395 - loss: 0.6134
Epoch 3/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 668s 1s/step - accuracy: 0.7258 - loss: 0.5944
Epoch 4/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 661s 1s/step - accuracy: 0.7430 - loss: 0.5735
Epoch 5/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 660s 1s/step - accuracy: 0.7216 - loss: 0.5917
Epoch 6/10
317/600 ━━━━━━━━━━━━━━━━━━━━ 36:08 8s/step - accuracy: 0.7576 - loss: 0.5559

In [ ]:
def nine_layer_cnn_softmax_kernel5():
  model = keras.models.Sequential([
      resize_rescale,
    data_enhance,
    keras.layers.Conv2D(filters=64, kernel_size=(5,5), activation='relu', input_shape=(650,650,3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(filters=128, kernel_size=(5,5), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(2, activation='softmax')
      ])
  model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [ ]:
model2=nine_layer_cnn_softmax_kernel5()
model2.fit(train_imgs,epochs=10)
y_pred_probs = model2.predict(test_imgs.map(lambda x, y: x))
y_pred = np.argmax(y_pred_probs, axis=1)
accuracy = accuracy_score(test_labels, y_pred)
f1 = f1_score(test_labels, y_pred)
precision = precision_score(test_labels, y_pred)
recall = recall_score(test_labels, y_pred)
print("Model 2")
print(accuracy)
print(f1)
print(precision)
print(recall)

In [ ]:
def densenet121():
    base_model=tf.keras.applications.DenseNet121(input_shape=(650,650,3),
                                                include_top=False,
                                                weights='imagenet')
    base_model.trainable=False
    model=tf.keras.models.Sequential([
        base_model,
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Dense(512,activation="relu"),
        keras.layers.Dense(2,activation='softmax')
    ])
    model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    return model

In [ ]:
model3=densenet121()
model3.fit(train_imgs,epochs=10)
y_pred_probs = model3.predict(test_imgs.map(lambda x, y: x))
y_pred = np.argmax(y_pred_probs, axis=1)
accuracy = accuracy_score(test_labels, y_pred)
f1 = f1_score(test_labels, y_pred)
precision = precision_score(test_labels, y_pred)
recall = recall_score(test_labels, y_pred)
print("Model")
print(accuracy)
print(f1)
print(precision)
print(recall)

In [ ]:
from sklearn.ensemble import VotingClassifier

In [ ]:
v1=VotingClassifier(estimators=[('model 1',model1),('model 2',model2)],voting="soft")
v2=VotingClassifier(estimators=[('model 3',model3),('model 2',model2)],voting="soft")
vc3soft=VotingClassifier(estimators=[('v1',v1),('v2',v2)],voting="soft")
model4=vc3soft.fit(train_imgs, train_labels,epochs=15)
y_pred_probs = model4.predict(test_imgs.map(lambda x, y: x))
y_pred = np.argmax(y_pred_probs, axis=1)
accuracy = accuracy_score(test_labels, y_pred)
f1 = f1_score(test_labels, y_pred)
precision = precision_score(test_labels, y_pred)
recall = recall_score(test_labels, y_pred)
print("Model4")
print(accuracy)
print(f1)
print(precision)
print(recall)

In [ ]:
v1=VotingClassifier(estimators=[('model 1',model1),('model 2',model2)],voting="hard")
v2=VotingClassifier(estimators=[('model 3',model3),('model 2',model2)],voting="hard")
vc3soft=VotingClassifier(estimators=[('v1',v1),('v2',v2)],voting="hard")
model5=vc3soft.fit(train_imgs,train_labels,epochs=15)
y_pred_probs = model5.predict(test_imgs.map(lambda x, y: x))
y_pred = np.argmax(y_pred_probs, axis=1)
accuracy = accuracy_score(test_labels, y_pred)
f1 = f1_score(test_labels, y_pred)
precision = precision_score(test_labels, y_pred)
recall = recall_score(test_labels, y_pred)
print("Model5")
print(accuracy)
print(f1)
print(precision)
print(recall)

In [ ]:
import pickle


In [ ]:

with open('model.pkl', 'wb') as model_file:
  pickle.dump(model, model_file)

In [ ]:
model=________

In [ ]:
! pip install -q gradio
with open('model.pkl','rb') as file:
  model=pickle.load(file)

In [ ]:
interface = gr.Interface(
    fn=prediction,
    inputs=[
gr.inpus.Image(type="filepath"),
gr.inputs.Textbox(label="Location")],
    outputs=[gr.outputs.Textbox(num_top_classes=2,label="Prediction By Model"),
gr.outputs.Markdown(label="Maps")],
    title="Brain Stroke Prediction with  MRI Scans using Ensemble Machine Learning"
)
interface.launch(share=True)

